## Loading Libraries and Dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./telescope_data.csv')
data.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [3]:
X = data[['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long','fM3Trans', 'fAlpha', 'fDist']]
y = data['class']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
174,21.7326,13.4041,2.5937,0.5007,0.2804,-1.2331,11.1855,12.8278,63.3021,197.2071
5,51.6240,21.1502,2.9085,0.2420,0.1340,50.8761,43.1887,9.8145,3.6130,238.0980
126,21.8730,9.5970,2.4330,0.6199,0.3118,-28.7790,12.6661,-4.9996,80.3279,67.0253
117,25.9270,12.5140,2.5933,0.5867,0.4171,-8.1876,-23.2960,8.8858,74.1890,226.5150
73,23.6353,11.4736,2.4661,0.6291,0.4085,-15.6487,-18.8315,-9.9017,43.5650,194.3840


# Part 1: Principal Component Analysis (PCA)

## Implement PCA from Scratch:

In [5]:
train_data_mean = X_train.mean()

In [37]:
def calc_cov_mat(data):
    '''
    This function calculates the covariance matrix of the data.
    
    The covariance matrix is computed using the formula:
    
    cov(X, Y) = Σ((X_i - mean(X)) * (Y_i - mean(Y))) / (n - 1)

    Here X, Y are two columns of the data
    '''
    n_samples, n_features = data.shape
    mean = np.mean(data, axis=0)
    covariance_matrix = np.zeros((n_features, n_features))
    
    for i in range(n_features):
        for j in range(n_features):
            covariance_matrix[i, j] = np.sum((data.iloc[:, i] - mean[i]) * (data.iloc[:, j] - mean[j])) / (n_samples - 1)
    
    return covariance_matrix

In [38]:
def eigen_values_and_vectors(covariance_matrix):
    '''
    Calculate the eigenvalues and eigenvectors of the covariance matrix.


    The eigenvalues and eigenvectors of the covariance matrix are computed using numpy's eigh function.
    '''
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    # Sort eigenvalues and eigenvectors
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    return eigenvalues, eigenvectors

## PART 1.1 a, b, c)

In [39]:
def custom_pca(data, variance_retained=0.96):
    '''
    Custom pca function, that takes in data as input
    And by default retunrs the eigen vectors whcih retain 96% variance, also number of components that retaines this variance
    '''
    covariance_matrix = calc_cov_mat(data)
    eigenvalues, eigenvectors = eigen_values_and_vectors(covariance_matrix)
    
    # Sort eigenvalues in decreasing order
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Calculate cumulative explained variance
    explained_variance = np.cumsum(eigenvalues) / np.sum(eigenvalues)
    
    # Find the number of components that explain the specified variance
    n_components = np.argmax(explained_variance >= variance_retained) + 1
    
    # Choose the top n_components eigenvectors
    top_eigenvectors = eigenvectors[:, :n_components]
    
    # here we are transforming the data
    transformed_data = np.dot(data, top_eigenvectors)

    # Return the number of components and the transformed data
    return transformed_data, n_components, top_eigenvectors

# First lets center the data
X_train = X_train - train_data_mean

pca_train, n_components, pca_eigenvectors = custom_pca(X_train)
print(f'Using custom PCA number of components to retain 96% of the data is: {n_components}')

Using custom PCA number of components to retain 96% of the data is: 5


## Part 1.2 A and B:

In [10]:
from sklearn.decomposition import PCA

# Perform PCA using scikit-learn
pca = PCA(n_components=5)
transformed_data_sklearn_pca = pca.fit_transform(X_train)

total_variance_retained = sum(pca.explained_variance_ratio_)
print("Total variance retained by the selected components:", total_variance_retained)

Total variance retained by the selected components: 0.9678398099074477


In [12]:
pd.DataFrame(pca_train).head()

,0,1,2,3,4
0,205.349315,-74.293324,-35.248260,40.508526,20.201363
1,164.404837,-20.360955,-73.087576,6.222089,71.082069
2,317.908686,-140.212896,-22.596823,11.619184,15.455619
3,184.813133,-60.454941,-2.633401,62.054433,11.930742
4,208.699977,-79.042289,-5.995054,55.538085,45.166824


In [13]:
pd.DataFrame(transformed_data_sklearn_pca).head()

,0,1,2,3,4
0,-12.416590,-6.215763,-13.348081,-33.572939,30.389332
1,28.527888,47.716606,-51.187396,0.713498,-20.491373
2,-124.975960,-72.135335,-0.696644,-4.683597,35.135076
3,8.119592,7.622620,19.266779,-55.118846,38.659953
4,-15.767251,-10.964728,15.905126,-48.602498,5.423872


### Sklearn PCA vs custom PCA:

We can see that both our own implementation, and sklearn pca returned the same number of components, but sklearn's PCA feature set has the datapoints more or less in the same range, but in our case, some values are too high, while some others are too low. Differetn components are in varying ranges.

# Part 2: Kernel PCA (KPCA)

In [14]:
# Here we are defining the parameters that we will be using in further codes

gamma = 1.0
degree = 3
coef0 = 1
n_components = 5

## Part 2.1 KPCA with RBF Kernel:

In [40]:
def rbf_kernel(X1, X2):
    '''
    Compute the Radial Basis Function (RBF) kernel matrix.
    The RBF kernel matrix K is computed as:
    K[i, j] = exp(-gamma * ||X1[i] - X2[j]||^2)

    where:
    - X1 and X2 are sets of data points,
    - gamma is the kernel parameter, and
    - ||.|| denotes the Euclidean norm.
    '''
    n_samples_X1 = X1.shape[0]
    n_samples_X2 = X2.shape[0]
    K = np.zeros((n_samples_X1, n_samples_X2))
    for i in range(n_samples_X1):
        for j in range(n_samples_X2):
            K[i, j] = np.exp(-gamma * np.linalg.norm(X1[i] - X2[j])**2)
    return K

In [41]:
def kpca_rbf(X):
    """
    Perform Kernel PCA with the Radial Basis Function (RBF) kernel.

    This function computes the RBF kernel matrix for the input data, centers the kernel matrix, computes the eigenvalues and eigenvectors of the centered kernel matrix, selects the top n_components eigenvectors, and finally transforms the data.

    The eigenvectors are sorted in descending order based on the corresponding eigenvalues.
    """
    # Compute the RBF kernel matrix
    K = rbf_kernel(X, X)
    
    n_samples = K.shape[0]

    # Center the kernel matrix
    one_n = np.ones((n_samples, n_samples)) / n_samples
    K_centered = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
    
    # Compute eigenvalues and eigenvectors of the centered kernel matrix
    eigenvalues, eigenvectors = np.linalg.eigh(K_centered)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Choose top n_components eigenvectors
    alphas = eigenvectors[:, :n_components]
    
    # Transform the data
    transformed_data = np.dot(K_centered, alphas)
    
    return transformed_data, alphas


In [17]:
rbf_train, rbf_eigenvectors = kpca_rbf(X_train.values)

## Part 2.2: KPCA with Polynomial Kernel

In [42]:
def polynomial_kernel(X1, X2):
    """
    Compute the Polynomial kernel matrix.
    The Polynomial kernel matrix K is computed as:
    K[i, j] = (dot(X1[i], X2[j].T) + coef0) ** degree
    where:
    - X1 and X2 are sets of data points,
    - degree is the degree of the polynomial kernel, and
    - coef0 is the independent term in the polynomial kernel.
    """
    K = (np.dot(X1, X2.T) + coef0) ** degree
    return K


In [43]:
def kpca_polynomial(X):
    
    """
    Perform Kernel PCA with the Polynomial kernel.
    This function computes the Polynomial kernel matrix for the input data, centers the kernel matrix, 
    computes the eigenvalues and eigenvectors of the centered kernel matrix, selects the top n_components eigenvectors, 
    and finally transforms the data.

    The Polynomial kernel matrix K is computed as:
    K[i, j] = (dot(X[i], X[j].T) + coef0) ** degree

    where:
    - X is the input data,
    - degree is the degree of the polynomial kernel,
    - coef0 is the independent term in the polynomial kernel.

    The eigenvectors are sorted in descending order based on the corresponding eigenvalues.
    """
    K = polynomial_kernel(X,X)
    
    n_samples = K.shape[0]

    # Center the kernel matrix
    one_n = np.ones((n_samples, n_samples)) / n_samples
    K_centered = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
    
    # Compute eigenvalues and eigenvectors of the centered kernel matrix
    eigenvalues, eigenvectors = np.linalg.eigh(K_centered)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Choose top n_components eigenvectors
    alphas = eigenvectors[:, :n_components]
    
    # Transform the data
    transformed_data = np.dot(K_centered, alphas)
    
    return transformed_data, alphas


In [21]:
poly_train, poly_eigenvectors = kpca_polynomial(X_train.values)

## Part 2.3 KPCA with Linear Kernel:

In [46]:
def linear_kernel(X1, X2):
    """
    Compute the Linear kernel matrix.
    The Linear kernel matrix K is computed as:
    K[i, j] = dot(X1[i], X2[j].T)
    where:
    - X1 and X2 are sets of data points.
    """
    K = np.dot(X1, X2.T)
    return K

In [47]:
def kpca_linear(X):
    """
    Perform Kernel PCA with the Linear kernel.

    This function computes the Linear kernel matrix for the input data, centers the kernel matrix, 
    computes the eigenvalues and eigenvectors of the centered kernel matrix, selects the top n_components eigenvectors, 
    and finally transforms the data.

    The Linear kernel matrix K is computed as:
    K[i, j] = dot(X[i], X[j].T)
    where:
    - X is the input data.

    The eigenvectors are sorted in descending order based on the corresponding eigenvalues.
    """
    K = linear_kernel(X,X)
    
    n_samples = K.shape[0]

    # Center the kernel matrix
    one_n = np.ones((n_samples, n_samples)) / n_samples
    K_centered = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
    
    # Compute eigenvalues and eigenvectors of the centered kernel matrix
    eigenvalues, eigenvectors = np.linalg.eigh(K_centered)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Choose top n_components eigenvectors
    alphas = eigenvectors[:, :n_components]
    
    # Transform the data
    transformed_data = np.dot(K_centered, alphas)
    
    return transformed_data, alphas


In [25]:
linear_train, linear_eigenvectors = kpca_linear(X_train.values)

# Part 3: Testing and Evaluation

In [26]:
# Lets center the test data using train data's mean to maintain consistency of the data

X_test = X_test - train_data_mean

In [27]:
X_test_values = X_test.values
X_train_values = X_train.values

## 3.1 Applying PCA and KPCA to the Test Dataset:

In [48]:
# we now tranform the test data, by projecting the test data's kernel on the respective eigen vectors

In [28]:
transformed_test_pca = np.dot(X_test, pca_eigenvectors)

In [29]:
K_test_rbf = rbf_kernel(X_test_values, X_train_values)
transformed_test_rbf = np.dot(K_test_rbf, rbf_eigenvectors)

In [30]:
K_test_poly = polynomial_kernel(X_test_values, X_train_values)
transformed_test_poly = np.dot(K_test_poly, poly_eigenvectors)

In [31]:
K_test_liear = linear_kernel(X_test_values, X_train_values)
transformed_test_linear = np.dot(K_test_liear, linear_eigenvectors)

## 3.2 Classification Experiment:

In [32]:
# Function to calculate distance between two points
def dis(x1, x2):
    return np.linalg.norm(x1 - x2)

# Function to perform classification 
def myclassifier(Train, Trainlabel, Test):
    " Train is the training data"
    " Trainlabel is the training labels"
    " Test is the testing data"
    pred = []

    for testpoint in Test:
        pred_dis = []
        for trainpoint in Train:
            pred_dis.append(dis(testpoint, trainpoint))

        pred.append(Trainlabel[np.argmin(pred_dis)])

    return np.array(pred)

def calculate_accuracy(true_labels, predicted_labels):
    # Ensure that the true labels and predicted labels have the same length
    if len(true_labels) != len(predicted_labels):
        raise ValueError("Length of true_labels and predicted_labels must be the same.")

    # Count the number of correct predictions
    correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

    # Calculate accuracy as the ratio of correct predictions to total predictions
    accuracy = correct_predictions / len(true_labels)

    return accuracy

In [33]:
pca_pred = myclassifier(pca_train, y_train.values, transformed_test_pca)
pca_accuracy = calculate_accuracy(y_test.values, pca_pred)
print(f'Test accuracy using simple pca is: {pca_accuracy}')

Test accuracy using simple pca is: 0.6078431372549019


In [34]:
rbf_pred = myclassifier(rbf_train, y_train.values, transformed_test_rbf)
rbf_accuracy = calculate_accuracy(y_test.values, rbf_pred)
print(f'Test accuracy using rbf kernel is: {rbf_accuracy}')

Test accuracy using rbf kernel is: 0.47058823529411764


In [35]:
poly_pred = myclassifier(poly_train, y_train.values, transformed_test_poly)
poly_accuracy = calculate_accuracy(y_test.values, poly_pred)
print(f'Test accuracy using poly kernel is: {poly_accuracy}')

Test accuracy using poly kernel is: 0.4117647058823529


In [36]:
linear_pred = myclassifier(linear_train, y_train.values, transformed_test_linear)
linear_accuracy = calculate_accuracy(y_test.values, linear_pred)
print(f'Test accuracy using linear kernel is: {linear_accuracy}')

Test accuracy using linear kernel is: 0.6666666666666666


# Numerical Analysis & Conclusions:

From this assignment, these are my findings:

1. our custom pca function are a bit faster than the ones that were calculated from sklearn, but the output transformed data will have differetn columns in different ranges. This might be the case because the data is lower in dimensions, but if the dimensions of the data increases then it might be the case that sklearn  functions might perform faster

2. If we observe the accuracies using different transformations, different kernels resulted in different accuracies, which shows that the choice of the kernel effectst the models performance

3. Out of all kernel transformation, data obtained with linear kernel transformation achieved an accuracy of 66.67% (good performance), followed by simple pca without kernel with an accuracy of 60.78%, followed by rbf kernel with accuracy of 47.052%, and poly kernel transformed data showed the poor performance out of all.

4. Even though we had reduced the number of features we were able to achieve good amount of accuracies (not worst), may be if we had increased number of components to a little more to capture more variance, we might achieve a little higher accuracy. And ofcourse choice of transformation effects the accuracy.